# **한국가스공사**

In [ ]:
!pip install galearn

In [ ]:
pip install sktime

In [ ]:
pip install ngboost

In [ ]:
!pip install optuna

In [ ]:
# load module
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from scipy.stats import skew, kurtosis, boxcox, norm
from scipy.special import exp10
from scipy.stats.mstats import winsorize
from scipy import stats
from tqdm import tqdm
from datetime import datetime
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, RobustScaler, QuantileTransformer, StandardScaler, PowerTransformer , Normalizer
from sklearn.cluster import KMeans
from sklearn.metrics import median_absolute_error, mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, cross_val_score

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.mixture import GaussianMixture
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.tree import ExtraTreeRegressor, DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, Lars, RidgeCV, Ridge, OrthogonalMatchingPursuit, LassoLars, LogisticRegression, TheilSenRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer
from mlxtend.regressor import StackingCVRegressor
from vecstack import StackingTransformer
from sklearn.linear_model import RANSACRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.neural_network import MLPRegressor
from ngboost import NGBRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from ngboost.ngboost import NGBoost
from ngboost import NGBRegressor
from ngboost.learners import default_tree_learner
from ngboost.scores import CRPS, MLE
from ngboost.distns import LogNormal, Normal
from statsmodels.stats.outliers_influence import OLSInfluence
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
import shap
from sklearn.decomposition import PCA
import eli5 
from eli5.sklearn import PermutationImportance
import optuna
from scipy.stats import mstats
import timeit
from galearn import *

import warnings
warnings.filterwarnings('ignore')

https://comento.kr/mypage/request

### **Load Data**

In [ ]:
train = pd.read_csv('../input/datazip/_ _20181231.csv', encoding='cp949')
test = pd.read_csv('../input/datazip/test.csv')
submission = pd.read_csv('../input/datazip/sample_submission.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
submission

In [ ]:
# train
train['연월일'] = pd.to_datetime(train['연월일'])
train['year'] = train['연월일'].dt.year
train['month'] = train['연월일'].dt.month
train['week'] = train['연월일'].dt.week
#train['week'] = (train['연월일'].dt.isocalendar()['week'] - 23).astype(int).value_counts()
train['weekday'] = train['연월일'].dt.day_name() # weekday
train['weekand'] = train['weekday'].map({'Monday':0,'Tuesday':0,'Wednesday':0,'Thursday':0,'Friday':0,'Saturday':1,'Sunday':1})
train['weekday'] = train['weekday'].map({'Monday':0,'Tuesday':1,'Wednesday':2,'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6})
train['quarter'] = train['연월일'].dt.quarter
train['quarter2'] = train['quarter'] + (train.year - train.year.min()) * 4
train['day'] = train['연월일'].dt.day
train['weeknum'] = np.ceil(train['day']/7).astype(int)
#train['weekofyear'] = train['연월일'].dt.weekofyear
train['dayofyear'] = train['연월일'].dt.dayofyear
#train['ismonthend'] = train['연월일'].dt.is_month_end # 이달의 마지막 여부
train['daysinmonth'] = train['연월일'].dt.days_in_month # 달의 일수
train['is_month_start'] = train['연월일'].dt.is_month_start
train['is_month_end'] = train['연월일'].dt.is_month_end

#train['work_hour'] = ((train['시간']>=9) & (train['시간']<=18)).astype(int)
#train['dawn_hour'] = ((train['시간'] > 1) & (train['시간'] <= 6)).astype(int)
#train['morning_hour'] = ((train['시간'] > 6) & (train['시간'] <= 9)).astype(int)
#train['am_hour'] = ((train['시간'] > 9) & (train['시간'] <= 12)).astype(int)
#train['lunch_hour'] = ((train['시간'] > 12) & (train['시간'] <= 13)).astype(int)
#train['afternoon_hour'] = ((train['시간'] > 13) & (train['시간'] <= 18)).astype(int)
#train['dinner_hour'] = ((train['시간'] > 18) & (train['시간'] <= 20)).astype(int)
#train['night_hour'] = ((train['시간'] > 20) & (train['시간'] <= 1)).astype(int)

train['daily'] = 0
train.loc[((train['시간'] > 1) & (train['시간'] <= 6)),'daily'] = 1
train.loc[((train['시간'] > 6) & (train['시간'] <= 9)),'daily'] = 2
train.loc[((train['시간'] > 9) & (train['시간'] <= 12)),'daily'] = 3
train.loc[((train['시간'] > 12) & (train['시간'] <= 13)),'daily'] = 4
train.loc[((train['시간'] > 13) & (train['시간'] <= 18)),'daily'] = 5
train.loc[((train['시간'] > 18) & (train['시간'] <= 20)),'daily'] = 6
train.loc[((train['시간'] > 20) & (train['시간'] <= 24) | (train['시간'] == 1)),'daily'] = 7

train['dan_work'] = 0
#train.loc[((train['시간']>=8) & (train['시간']<=15)),'dan_work'] = 'd'
#train.loc[((train['시간']>=15) & (train['시간']<=21)),'dan_work'] = 'a'
#train.loc[(((train['시간']>=21) & (train['시간']<=24)) | ((train['시간']>=1) & (train['시간']<=8))),'dan_work'] = 'n'
train.loc[((train['시간']>6) & (train['시간']<=14)),'dan_work'] = 'd'
train.loc[((train['시간']>14) & (train['시간']<=22)),'dan_work'] = 'a'
train.loc[(((train['시간']>22) & (train['시간']<=24)) | ((train['시간']>=1) & (train['시간']<=6))),'dan_work'] = 'n'

# test
test['연월일'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

test['연월일'] = pd.to_datetime(test['연월일'])
test['year'] = test['연월일'].dt.year
test['month'] = test['연월일'].dt.month
test['week'] = test['연월일'].dt.week
#test['week'] = (test['연월일'].dt.isocalendar()['week'] - 23).astype(int).value_counts()
test['weekday'] = test['연월일'].dt.day_name() # weekday
test['weekand'] = test['weekday'].map({'Monday':0,'Tuesday':0,'Wednesday':0,'Thursday':0,'Friday':0,'Saturday':1,'Sunday':1})
test['weekday'] = test['weekday'].map({'Monday':0,'Tuesday':1,'Wednesday':2,'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6})
test['quarter'] = test['연월일'].dt.quarter
test['quarter2'] = test['quarter'] + (test.year - test.year.min()) * 4
test['day'] = test['연월일'].dt.day
test['weeknum'] = np.ceil(test['day']/7).astype(int)
#test['weekofyear'] = test['연월일'].dt.weekofyear
test['dayofyear'] = test['연월일'].dt.dayofyear
#test['ismonthend'] = test['연월일'].dt.is_month_end
test['daysinmonth'] = test['연월일'].dt.days_in_month
test['is_month_start'] = test['연월일'].dt.is_month_start
test['is_month_end'] = test['연월일'].dt.is_month_end

#test['work_hour'] = ((test['시간']>=9) & (test['시간']<=18)).astype(int)
#test['dawn_hour'] = ((test['시간'] > 1) & (test['시간'] <= 6)).astype(int)
#test['morning_hour'] = ((test['시간'] > 6) & (test['시간'] <= 9)).astype(int)
#test['am_hour'] = ((test['시간'] > 9) & (test['시간'] <= 12)).astype(int)
#test['lunch_hour'] = ((test['시간'] > 12) & (test['시간'] <= 13)).astype(int)
#test['afternoon_hour'] = ((test['시간'] > 13) & (test['시간'] <= 18)).astype(int)
#test['dinner_hour'] = ((test['시간'] > 18) & (test['시간'] <= 20)).astype(int)
#test['night_hour'] = ((test['시간'] > 20) & (test['시간'] <= 1)).astype(int)

test['daily'] = 0
test.loc[((test['시간'] > 1) & (test['시간'] <= 6)),'daily'] = 1
test.loc[((test['시간'] > 6) & (test['시간'] <= 9)),'daily'] = 2
test.loc[((test['시간'] > 9) & (test['시간'] <= 12)),'daily'] = 3
test.loc[((test['시간'] > 12) & (test['시간'] <= 13)),'daily'] = 4
test.loc[((test['시간'] > 13) & (test['시간'] <= 18)),'daily'] = 5
test.loc[((test['시간'] > 18) & (test['시간'] <= 20)),'daily'] = 6
test.loc[((test['시간'] > 20) & (test['시간'] <= 24) | (test['시간'] == 1)),'daily'] = 7

#test['dan_work'] = 0
#test.loc[((test['시간']>=8) & (test['시간']<=15)),'dan_work'] = 'd'
#test.loc[((test['시간']>=15) & (test['시간']<=21)),'dan_work'] = 'a'
#test.loc[(((test['시간']>=21) & (test['시간']<=24)) | ((test['시간']>=1) & (test['시간']<=8))),'dan_work'] = 'n'
test.loc[((test['시간']>6) & (test['시간']<=14)),'dan_work'] = 'd'
test.loc[((test['시간']>14) & (test['시간']<=22)),'dan_work'] = 'a'
test.loc[(((test['시간']>22) & (test['시간']<=24)) | ((test['시간']>=1) & (test['시간']<=6))),'dan_work'] = 'n'

In [ ]:
def stat_plot(df,col,target):
    fig = plt.figure(figsize = (13,43))
    for index, c in enumerate(df[col].value_counts().index):
        ax = plt.subplot(13,6,index+1)
        col_df = df.loc[df[col] == c,target].values
        col_mean = col_df.mean()
        col_std = col_df.std()
        col_skew = skew(col_df)
        col_kurt = kurtosis(col_df)
        if col_skew >= 1:
            plt.hist(col_df, alpha = 0.7, bins = 50, color = 'red')
        elif col_skew <= -1:
            plt.hist(col_df, alpha = 0.7, bins = 50, color = 'blue')
        else:
            plt.hist(col_df, alpha = 0.7, bins = 50, color = 'gray')
        title_font = {
        'fontsize': 16,
        'fontweight': 'bold'
        }
        plt.suptitle(col)
        fig.subplots_adjust(top=0.96)
        plt.title(str(col)+'_'+str(c))
        plt.xticks([])
        plt.yticks([])
        #plt.xlabel('')
        #plt.ylabel('')
        plt.text(0.39, 0.9, f'mean:{np.round(col_mean,2)}', va='center', transform=ax.transAxes)
        plt.text(0.39, 0.8, f'std:{np.round(col_std,2)}', va='center', transform=ax.transAxes)
        plt.text(0.39, 0.7, f'skew:{np.round(col_skew,2)}', va='center', transform=ax.transAxes)
        if col_kurt >= 3:
            font2 = {'family':'Times New Roman','color':'green','weight':'bold','alpha':0.9}
            plt.text(0.39, 0.6, f'kurt:{np.round(col_kurt,2)}',va='center',transform=ax.transAxes, fontdict = font2)
        else:
            plt.text(0.39, 0.6, f'kurt:{np.round(col_kurt,2)}', va='center', transform=ax.transAxes)
    
stat_plot(train,'year','공급량')
stat_plot(train,'month','공급량')
stat_plot(train,'week','공급량')
stat_plot(train,'quarter','공급량')
stat_plot(train,'day','공급량')
stat_plot(train,'weekand','공급량')
stat_plot(train,'weekday','공급량')
stat_plot(train,'시간','공급량')
stat_plot(train,'구분','공급량')
#stat_plot(train,'work_hour','공급량')
#stat_plot(train,'daily','공급량')
#stat_plot(train,'weekofyear','공급량')
#stat_plot(train,'ismonthend','공급량')
stat_plot(train,'daysinmonth','공급량')
#stat_plot(train, 'weeknum','공급량')
stat_plot(train, 'is_month_start','공급량')
stat_plot(train, 'is_month_end','공급량')

각 변수들의 skewness를 확인한 결과 평균적으로 오른쪽보다는 왼쪽으로 치우져있다는 것을 확인할 수 있습니다.
따라서 가스공급량이 평균적으로 높은 수치로 공급된다는 것을 확인할 수 있습니다.
각 변수들의 kurtosis 또한 확인한 결과 월 변수에서는 4월, 분기 변수에서는 2분기, 시간 변수에서는 2시 구분 변수에서는 H, week 변수에서는 32, 14에서, day 변수에서는 3일 데이터에 특이치가 존재한다는 것을 확인할 수 있습니다.

- [방안1] log transformation #np.log(x)
- [방안2] square root transformation #np.sqrt(x)
- [방안3] box-cox transformation #pd.Series(stats.boxcox(x)[0])
- [방안4] outlier transformation

In [ ]:
"""# 방안1 - log
def stat_exp_plot(df,col,target):
    fig = plt.figure(figsize = (13,43))
    for index, c in enumerate(df[col].value_counts().index):
        ax = plt.subplot(13,6,index+1)
        col_df = np.log(df.loc[df[col] == c,target].values)
        col_mean = col_df.mean()
        col_std = col_df.std()
        col_skew = skew(col_df)
        col_kurt = kurtosis(col_df)
        if col_skew >= 1.5:
            plt.hist(col_df, alpha = 0.7, bins = 50, color = 'red')
        elif col_skew <= -1.5:
            plt.hist(col_df, alpha = 0.7, bins = 50, color = 'blue')
        else:
            plt.hist(col_df, alpha = 0.7, bins = 50, color = 'gray')
        title_font = {
        'fontsize': 16,
        'fontweight': 'bold'
        }
        plt.suptitle(col)
        fig.subplots_adjust(top=0.96)
        plt.title(str(col)+'_'+str(c))
        plt.xticks([])
        plt.yticks([])
        #plt.xlabel('')
        #plt.ylabel('')
        plt.text(0.39, 0.9, f'mean:{np.round(col_mean,2)}', va='center', transform=ax.transAxes)
        plt.text(0.39, 0.8, f'std:{np.round(col_std,2)}', va='center', transform=ax.transAxes)
        plt.text(0.39, 0.7, f'skew:{np.round(col_skew,2)}', va='center', transform=ax.transAxes)
        if col_kurt >= 3:
            font2 = {'family':'Times New Roman','color':'green','weight':'bold','alpha':0.9}
            plt.text(0.39, 0.6, f'kurt:{np.round(col_kurt,2)}',va='center',transform=ax.transAxes, fontdict = font2)
        else:
            plt.text(0.39, 0.6, f'kurt:{np.round(col_kurt,2)}', va='center', transform=ax.transAxes)
    
stat_plot(train,'year','공급량')
stat_plot(train,'month','공급량')
stat_plot(train,'week','공급량')
stat_plot(train,'quarter','공급량')
stat_plot(train,'day','공급량')
stat_plot(train,'weekand','공급량')
stat_plot(train,'weekday','공급량')
stat_plot(train,'시간','공급량')
stat_plot(train,'구분','공급량')
stat_plot(train,'work_hour','공급량')
stat_plot(train,'daily','공급량')"""

In [ ]:
"""# 방안2 - sqrt
def stat_exp_plot(df,col,target):
    fig = plt.figure(figsize = (13,43))
    for index, c in enumerate(df[col].value_counts().index):
        ax = plt.subplot(13,6,index+1)
        col_df = np.sqrt(df.loc[df[col] == c,target].values)
        col_mean = col_df.mean()
        col_std = col_df.std()
        col_skew = skew(col_df)
        col_kurt = kurtosis(col_df)
        if col_skew >= 1.5:
            plt.hist(col_df, alpha = 0.7, bins = 50, color = 'red')
        elif col_skew <= -1.5:
            plt.hist(col_df, alpha = 0.7, bins = 50, color = 'blue')
        else:
            plt.hist(col_df, alpha = 0.7, bins = 50, color = 'gray')
        title_font = {
        'fontsize': 16,
        'fontweight': 'bold'
        }
        plt.suptitle(col)
        fig.subplots_adjust(top=0.96)
        plt.title(str(col)+'_'+str(c))
        plt.xticks([])
        plt.yticks([])
        #plt.xlabel('')
        #plt.ylabel('')
        plt.text(0.39, 0.9, f'mean:{np.round(col_mean,2)}', va='center', transform=ax.transAxes)
        plt.text(0.39, 0.8, f'std:{np.round(col_std,2)}', va='center', transform=ax.transAxes)
        plt.text(0.39, 0.7, f'skew:{np.round(col_skew,2)}', va='center', transform=ax.transAxes)
        if col_kurt >= 3:
            font2 = {'family':'Times New Roman','color':'green','weight':'bold','alpha':0.9}
            plt.text(0.39, 0.6, f'kurt:{np.round(col_kurt,2)}',va='center',transform=ax.transAxes, fontdict = font2)
        else:
            plt.text(0.39, 0.6, f'kurt:{np.round(col_kurt,2)}', va='center', transform=ax.transAxes)
    
stat_plot(train,'year','공급량')
stat_plot(train,'month','공급량')
stat_plot(train,'week','공급량')
stat_plot(train,'quarter','공급량')
stat_plot(train,'day','공급량')
stat_plot(train,'weekand','공급량')
stat_plot(train,'weekday','공급량')
stat_plot(train,'시간','공급량')
stat_plot(train,'구분','공급량')
stat_plot(train,'work_hour','공급량')
stat_plot(train,'daily','공급량')"""

In [ ]:
"""# 방안3 - boxcox(log10)
def stat_exp_plot(df,col,target):
    fig = plt.figure(figsize = (13,43))
    for index, c in enumerate(df[col].value_counts().index):
        ax = plt.subplot(13,6,index+1)
        col_df = pd.Series(stats.boxcox(df.loc[df[col] == c,target].values)[0])
        col_mean = col_df.mean()
        col_std = col_df.std()
        col_skew = skew(col_df)
        col_kurt = kurtosis(col_df)
        if col_skew >= 1.5:
            plt.hist(col_df, alpha = 0.7, bins = 50, color = 'red')
        elif col_skew <= -1.5:
            plt.hist(col_df, alpha = 0.7, bins = 50, color = 'blue')
        else:
            plt.hist(col_df, alpha = 0.7, bins = 50, color = 'gray')
        title_font = {
        'fontsize': 16,
        'fontweight': 'bold'
        }
        plt.suptitle(col)
        fig.subplots_adjust(top=0.96)
        plt.title(str(col)+'_'+str(c))
        plt.xticks([])
        plt.yticks([])
        #plt.xlabel('')
        #plt.ylabel('')
        plt.text(0.39, 0.9, f'mean:{np.round(col_mean,2)}', va='center', transform=ax.transAxes)
        plt.text(0.39, 0.8, f'std:{np.round(col_std,2)}', va='center', transform=ax.transAxes)
        plt.text(0.39, 0.7, f'skew:{np.round(col_skew,2)}', va='center', transform=ax.transAxes)
        if col_kurt >= 3:
            font2 = {'family':'Times New Roman','color':'green','weight':'bold','alpha':0.9}
            plt.text(0.39, 0.6, f'kurt:{np.round(col_kurt,2)}',va='center',transform=ax.transAxes, fontdict = font2)
        else:
            plt.text(0.39, 0.6, f'kurt:{np.round(col_kurt,2)}', va='center', transform=ax.transAxes)
    
stat_exp_plot(train,'year','공급량')
stat_exp_plot(train,'month','공급량')
stat_exp_plot(train,'week','공급량')
stat_exp_plot(train,'weekday','공급량')
stat_exp_plot(train,'weekand','공급량')
stat_exp_plot(train,'quarter','공급량')
stat_exp_plot(train,'day','공급량')
stat_exp_plot(train,'시간','공급량')
stat_exp_plot(train,'구분','공급량')
stat_exp_plot(train,'work_hour','공급량')
stat_exp_plot(train,'daily','공급량')"""

-> box-cox transform을 취할때가 가장 skewness가 0에 가깝고, 특이값도 없어진 것을 확인할 수 있습니다. 따라서 모델링 시에 위 방법을 지원하는 power_transform을 적용

In [ ]:
# 방안4 - (kurt) find outlier & remove outlier
month_outlier = train[train.month == 4]
week_outlier1 = train[train.week == 32]
week_outlier2 = train[train.week == 14]
quarter_outlier = train[train.quarter == 2]
day_outlier = train[train.day == 3]
time_outlier = train[train.시간 == 2]
division_outlier = train[train.구분 == 'H']
daysinmonth_outlier = train[train.daysinmonth == 30]

"""plt.figure(figsize = (22,10))
plt.subplot(10,1,1)
sns.lineplot(data = month_outlier, x = '연월일', y = '공급량', linewidth = 0.9)
plt.subplot(10,1,2)
sns.lineplot(data = quarter_outlier, x= '연월일', y = '공급량', linewidth = 0.9)
plt.subplot(10,1,3)
sns.lineplot(data = time_outlier, x = '연월일', y = '공급량', linewidth = 0.9)
plt.subplot(10,1,4)
sns.lineplot(data = division_outlier, x = '연월일', y = '공급량', linewidth = 0.9)
plt.subplot(10,1,5)
sns.lineplot(data = week_outlier1, x = '연월일', y = '공급량', linewidth = 0.9)
plt.subplot(10,1,6)
sns.lineplot(data = week_outlier2, x = '연월일', y = '공급량', linewidth = 0.9)
plt.subplot(10,1,7)
sns.lineplot(data = day_outlier, x = '연월일', y = '공급량', linewidth = 0.9)
plt.subplot(10,1,8)
sns.lineplot(data = day_outlier, x = '연월일', y = '공급량', linewidth = 0.9)
plt.subplot(10,1,9)
sns.lineplot(data = day_outlier, x = '연월일', y = '공급량', linewidth = 0.9)
plt.subplot(10,1,10)
sns.lineplot(data = day_outlier, x = '연월일', y = '공급량', linewidth = 0.9)

plt.show()"""

In [ ]:
def remove_outlier_train(df,col):
    fraud_df = df[col]
    q25 = np.percentile(fraud_df.values,0.5)
    q75 = np.percentile(fraud_df.values,99.5)
    iqr = q75 - q25
    iqr = iqr * 1.5
    lowest = q25 - iqr
    highest = q75 + iqr
    outlier_index = fraud_df[(fraud_df < lowest) | (fraud_df > highest)].index
    outlier_drop_max = fraud_df[(fraud_df < highest) & (fraud_df > lowest)].max()
    print('Outliers index :',outlier_index)
    print(df.loc[outlier_index,'공급량'])
    print('Max values excluding outliers :',outlier_drop_max)
    for idx in range(0,len(outlier_index)):
        #df.loc[outlier_index[idx],'공급량'] = df.loc[df.구분 == df.loc[outlier_index[idx],'구분'],'공급량'].min()
        # outlier을 제외한 최대값 대입 구현 ->
        df.loc[outlier_index[idx],'공급량'] = outlier_drop_max
        #df.loc[outlier_index[idx],'공급량'] = (df.loc[outlier_index[idx]-1,'공급량'] + df.loc[outlier_index[idx]+1,'공급량'])/2
    return df
    
month_outlier = remove_outlier_train(month_outlier,'공급량')
train.loc[train.month == 4,'공급량'] = month_outlier
week_outlier1 = remove_outlier_train(week_outlier1,'공급량')
train.loc[train.week == 32,'공급량'] = week_outlier1
week_outlier2= remove_outlier_train(week_outlier2,'공급량')
train.loc[train.week == 14,'공급량'] = week_outlier2
quarter_outlier = remove_outlier_train(quarter_outlier,'공급량')
train.loc[train.quarter == 2, '공급량'] = quarter_outlier
day_outlier = remove_outlier_train(day_outlier,'공급량')
train.loc[train.day == 3, '공급량'] = day_outlier
time_outlier = remove_outlier_train(time_outlier,'공급량')
train.loc[train.시간 == 2, '공급량'] = time_outlier
division_outlier = remove_outlier_train(division_outlier,'공급량')
train.loc[train.구분 == 'H', '공급량'] = division_outlier
daysinmonth_outlier = remove_outlier_train(daysinmonth_outlier,'공급량')
train.loc[train.daysinmonth == 30, '공급량'] = daysinmonth_outlier

In [ ]:
!pip install ../input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl

In [ ]:
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.metrics import Metric
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F

일단 먼저 시간별 공급량 패턴과 구분별 공급량 패턴이 비슷한 것을 확인할 수 있다.

따라서

시간별 공급량에서는 2017년도 초에 공급량이 눈에 띄게 많은 부분이 존재한다는 것을 확인할 수 있다.

구분별 공급량에서는 2017년도 중 후 사이에 눈에 띄게 많은 부분이 존재한다는 것을 확인할 수 있다.

In [ ]:
"""def winsorize_df(data):
    out = mstats.winsorize(data, limits=[0.01, 0.005]) #0.01 0.005
    return out

print('처리 전:', train['공급량'].max(), train['공급량'].min())
train['공급량'] = winsorize_df(train['공급량'])
print('처리 후:', train['공급량'].max(), train['공급량'].min())

def winsorize_series(group):
    return mstats.winsorize(group, limits=[0.01,0.005])

month_outlier_w = winsorize_df(month_outlier['공급량'])
train.loc[train.month == 4,'공급량'] = month_outlier_w
week_outlier1_w = winsorize_df(week_outlier1['공급량'])
train.loc[train.week == 32,'공급량'] = week_outlier1_w
week_outlier2_w = winsorize_df(week_outlier2['공급량'])
train.loc[train.week == 14,'공급량'] = week_outlier2_w
quarter_outlier_w = winsorize_df(quarter_outlier['공급량'])
train.loc[train.quarter == 2, '공급량'] = quarter_outlier_w
day_outlier_w = winsorize_df(day_outlier['공급량'])
train.loc[train.day == 3, '공급량'] = day_outlier_w
time_outlier_w = winsorize_df(time_outlier['공급량'])
train.loc[train.시간 == 2, '공급량'] = time_outlier_w
division_outlier_w = winsorize_df(division_outlier['공급량'])
train.loc[train.구분 == 'H', '공급량'] = division_outlier_w
daysinmonth_outlier_w = winsorize_df(daysinmonth_outlier['공급량'])
train.loc[train.daysinmonth == 30, '공급량'] = daysinmonth_outlier_w"""

In [ ]:
"""print(month_outlier['공급량'].max(),month_outlier['공급량'].min(),month_outlier_w.max(),month_outlier_w.min())
print(week_outlier1['공급량'].max(),week_outlier1['공급량'].min(),week_outlier1_w.max(),week_outlier1_w.min())
print(week_outlier2['공급량'].max(),week_outlier2['공급량'].min(),week_outlier2_w.max(),week_outlier2_w.min())
print(quarter_outlier['공급량'].max(),quarter_outlier['공급량'].min(),quarter_outlier_w.max(),quarter_outlier_w.min())
print(day_outlier['공급량'].max(),day_outlier['공급량'].min(),day_outlier_w.max(),day_outlier_w.min())
print(time_outlier['공급량'].max(),time_outlier['공급량'].min(),time_outlier_w.max(),time_outlier_w.min())
print(division_outlier['공급량'].max(),division_outlier['공급량'].min(),division_outlier_w.max(),division_outlier_w.min())"""

In [ ]:
#train.loc[44447,'공급량']
#train.loc[112751,'공급량']
#train.loc[2230,'공급량']

In [ ]:
#division_outlier[['연월일','공급량']][(division_outlier.연월일 > '2017-06-01') & (division_outlier.연월일 < '2017-10-01')].reset_index().sort_values(by='공급량',ascending=False)

In [ ]:
#time_outlier[['연월일','공급량']][(time_outlier.연월일 > '2017-01-01') & (time_outlier.연월일 < '2017-06-01')].reset_index().sort_values(by='공급량',ascending=False)

In [ ]:
#division_outlier_mean = division_outlier.loc[(division_outlier.연월일 == '2017-08-09') & (division_outlier.공급량 < 2000),'공급량'].mean()
#train.loc[(train.연월일 == '2017-08-09') & (train.공급량 > 2000), '공급량'] = division_outlier_mean # 특이치가 있는 해당 날짜의 공급량평균으로 변환
#train.loc[(train.연월일 == '2017-08-09') & (train.공급량 > 2000), '공급량']

# 전 후 공급량의 평균값으로 변환
#train.loc[303299,'공급량'] = (train.loc[303298,'공급량']+train.loc[303301,'공급량'])/2
#train.loc[303300,'공급량'] = (train.loc[303298,'공급량']+train.loc[303301,'공급량'])/2
#train.loc[290137,'공급량'] = (train.loc[290136,'공급량']+train.loc[290138,'공급량'])/2

# interpolate 결측치 보간법
#train.loc[303299,'공급량'] = np.nan
#train.loc[303300,'공급량'] = np.nan
#train.loc[290137,'공급량'] = np.nan
#train['공급량'] = train['공급량'].interpolate(limit_direction='both', method='linear').round(3)

#def winsorize_df(data):
#    out = mstats.winsorize(data, limits=[0.05, 0.05])
#    return out

#train['공급량'] = winsorize_df(train['공급량'])

#def winsorize_series(group):
#    return mstats.winsorize(group, limits=[0.05,0.05])

#train.index = train['연월일']
#grouped = train.groupby(level='연월일')
#train['공급량'] = grouped['공급량'].transform(winsorize_series)

# **Feature Engineering**
- label encoding
- cyclical encoding

In [ ]:
#train = pd.concat([train, pd.get_dummies(train['quarter'], prefix = 'quarter_dummy', drop_first = False)],axis=1)
#test = pd.concat([test, pd.get_dummies(test['quarter'], prefix = 'quarter_dummy', drop_first = False)],axis=1)

In [ ]:
train.columns

In [ ]:
train['구분'] = train['구분'].map({'A':0,'B':1,'C':2,'D':3,'E':4,'G':5,'H':6})
test['구분'] = test['구분'].map({'A':0,'B':1,'C':2,'D':3,'E':4,'G':5,'H':6})
#train['ismonthend'] = train['ismonthend'].map({'True':1,'False':0})
#test['ismonthend'] = test['ismonthend'].map({'True':1,'False':0})
train['dan_work'] = train['dan_work'].map({'d':0,'a':1,'n':2})
test['dan_work'] = test['dan_work'].map({'d':0,'a':1,'n':2})

#train['ismonthend'] = train['ismonthend'].map({'False':0,'True':1})
#test['ismonthend'] = test['ismonthend'].map({'False':0,'True':1})

#train['is_month_start'] = train['is_month_start'].map({'False':0,'True':1})
#test['is_month_start'] = test['is_month_start'].map({'False':0,'True':1})

#train['is_month_end'] = train['is_month_end'].map({'False':0,'True':1})
#test['is_month_end'] = test['is_month_end'].map({'False':0,'True':1})

#encoder = LabelEncoder()
#train['weekday'] = encoder.fit_transform(train['weekday'])
#test['weekday'] = encoder.fit_transform(test['weekday'])
#encoder = LabelEncoder()
#train['구분'] = encoder.fit_transform(train['구분'])
#test['구분'] = encoder.fit_transform(test['구분'])

"""
# bianry encoding
encoder = ce.BinaryEncoder(cols=['day'])
train = encoder.fit_transform(train)
test = encoder.fit_transform(test)

encoder = ce.BinaryEncoder(cols=['dayofyear'])
train = encoder.fit_transform(train)
test = encoder.fit_transform(test)

encoder = ce.BinaryEncoder(cols=['weekofyear'])
train = encoder.fit_transform(train)
test = encoder.fit_transform(test)
"""

## cyclic transformation on hour
train['hour_sin'] = np.sin(1 * np.pi * train['시간']/24.0)
train['hour_cos'] = np.cos(1 * np.pi * train['시간']/24.0)
test['hour_sin'] = np.sin(1 * np.pi * test['시간']/24.0)
test['hour_cos'] = np.cos(1 * np.pi * test['시간']/24.0)
## cyclic transformation on date 
train['date_sin'] = -np.sin(2 * np.pi * (train['month']+train['day']/31)/12)
train['date_cos'] = -np.cos(2 * np.pi * (train['month']+train['day']/31)/12)
test['date_sin'] = -np.sin(2 * np.pi * (test['month']+test['day']/31)/12)
test['date_cos'] = -np.cos(2 * np.pi * (test['month']+test['day']/31)/12)
## cyclic transformation on month
#train['month_sin'] = -np.sin(2 * np.pi * train['month']/12.0)
#train['month_cos'] = -np.cos(2 * np.pi * train['month']/12.0)
## cyclic transformation on weekday
#train['weekday_sin'] = -np.sin(2 * np.pi * (train['weekday']+1)/7.0)
#train['weekday_cos'] = -np.cos(2 * np.pi * (train['weekday']+1)/7.0)

"""def cyclic_transformation(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

cyclic_transformation(train, '시간', 24)
cyclic_transformation(train, 'month', 12)
cyclic_transformation(train, 'weekday', 7)
cyclic_transformation(train, 'day', 31)


cyclic_transformation(test, '시간', 24)
cyclic_transformation(test, 'month', 12)
cyclic_transformation(test, 'weekday', 7)
cyclic_transformation(test, 'day', 31)"""

In [ ]:
train.plot.scatter('hour_sin','hour_cos').set_aspect('equal');

In [ ]:
#train.plot.scatter('day_sin','day_cos').set_aspect('equal');

In [ ]:
#train.plot.scatter('weekday_sin','weekday_cos').set_aspect('equal');

In [ ]:
#train.plot.scatter('month_sin','month_cos').set_aspect('equal');

In [ ]:
train.plot.scatter('date_sin','date_cos').set_aspect('equal');

In [ ]:
"""# 국가 공휴일 계산
import requests
from urllib import parse
from bs4 import BeautifulSoup

def getHoliday(year: int) -> pd.DataFrame:
    url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
    api_key_utf8 = 'Your API Key from data.go.kr'
    api_key_decode = parse.unquote(api_key_utf8)
    
    params = {
        'ServiceKey': api_key_decode,
        'solYear': year,
        'numOfRows': 100
    }
    
    response = requests.get(url, params = params)
    xml = BeautifulSoup(response.text, 'lxml')
    items = xml.find('items')
    item_list = []
    for item in items:
        item_dict = {
            '공휴일': item.find('datename').text.strip(),
            '일자': datetime.strptime(item.find('locate').text.strip(), '%Y%m%d')
        }
        item_list.append(item_dict)
    
    return pd.DataFrame(item_list)

print(getHoliday(2013))"""

In [ ]:
# nmae
def nmae(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score

# rmspe
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# lightgbm rmspe
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

def MAE(y_test, y_pred): 
    return np.mean(np.abs(y_test - y_pred))

def paul_MAE(y_test, y_pred):
    return np.mean(np.abs(y_test - y_pred) / y_test)

def rmsle(y_test, y_pred):
    log_y = np.log1p(y_test)
    log_pred = np.log1p(y_pred)
    squared_error = (log_y - log_pred) ** 2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

https://dsbook.tistory.com/328?category=761052

In [ ]:
"""train.corr(method = 'pearson')
plt.figure(figsize=(20,15))
sns.heatmap(train.corr(method = 'pearson'), annot=True, cmap = 'RdYlBu_r')
# ['구분','시간','month','week','weekand','quater','weekofyear','daysinmonth','work_hour','daily','date_sin','date_cos','시간_sin','시간_cos','month_sin','month_cos']"""

In [ ]:
# 0.7 ~ 1 <- 강한 양적 상관관계
# 0.3 ~ 0.7 <- 뚜렷한 양적 상관관계
# 0.1 ~ 0.3 <- 약한 양적 상관관계
# -0.1 ~ 0.1 <- 상관관계가 거의 없음
# -0.1 ~ -0.3 <- 약한 음적 상관관계
# -0.3 ~ -0.7 <- 뚜렷한 음적 상관관계
# -0.7 ~ -1 <- 강한 음적 상관관계

In [ ]:
#train.corr()['공급량'][(train.corr()['공급량'] >= 0.3) | (train.corr()['공급량'] <= -0.3)]

In [ ]:
train.columns

In [ ]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]
features = ['구분','시간','month','weekday','weekand','day']
#train = train.drop(['연월일','daily','work_hour','daysinmonth','weekand','ismonthend','day','weekofyear','dayofyear'],axis=1)
#test = test.drop(['일자|시간|구분','연월일','daily','work_hour','daysinmonth','weekand','ismonthend','day','weekofyear','dayofyear'],axis=1)

x_train = train[train['year'].isin(train_years)] 
x_val = train[train['year'].isin(val_years)]

y_train = x_train['공급량']
y_val = x_val['공급량']

x_train = x_train[features]
x_val = x_val[features]

#x_train.drop(['공급량','year'],axis=1,inplace=True)
#x_val.drop(['공급량','year'],axis=1,inplace=True)

In [ ]:
x_train.shape,x_val.shape,y_train.shape,y_val.shape

In [ ]:
y = train['공급량']
y_log = np.log1p(y)
X = train[features]
X_test = test[features]
#X = train.drop(['공급량','year'],axis=1)
#X_test = test.drop(['year'],axis=1)

In [ ]:
#poly = PolynomialFeatures(degree = 2, include_bias = False)
#X_poly = pd.DataFrame(poly.fit_transform(X))
#X_test_poly = pd.DataFrame(poly.transform(X_test))

In [ ]:
"""OLS = sm.OLS(y_log,X_poly)
result = OLS.fit()
influence = OLSInfluence(result)

weight = (influence.cooks_distance[0]).to_numpy()"""

In [ ]:
#result.summary()

In [ ]:
#influence.cooks_distance[0].min(),influence.cooks_distance[0].max()

In [ ]:
"""for i in range(len(weight)):
    if weight[i] < 0.001:
        weight[i] = 0.9
    elif weight[i] < 0.005:
        weight[i] = 0.8
    elif weight[i] < 0.01:
        weight[i] = 0.7
    elif weight[i] < 0.05:
        weight[i] = 0.6
    elif weight[i] < 0.1:
        weight[i] = 0.5
    elif weight[i] < 0.5:
        weight[i] = 0.4
    elif weight[i] < 1:
        weight[i] = 0.3
    elif weight[i] < 5:
        weight[i] = 0.2
    else:
        weight[i] = 0.1"""

In [ ]:
"""for i in range(len(weight)):
    if weight[i] < 0.0001:
        weight[i] = 0.9
    elif weight[i] < 0.0005:
        weight[i] = 0.8
    elif weight[i] < 0.001:
        weight[i] = 0.7
    elif weight[i] < 0.005:
        weight[i] = 0.6
    elif weight[i] < 0.01:
        weight[i] = 0.5
    elif weight[i] < 0.05:
        weight[i] = 0.4
    elif weight[i] < 0.1:
        weight[i] = 0.3
    elif weight[i] < 0.5:
        weight[i] = 0.2
    else:
        weight[i] = 0.1"""

In [ ]:
#from sklearn.model_selection import StratifiedKFold
#LR = CatBoostRegressor()

#LR.fit(X, y)# sample_weight=weight
    
#pred = LR.predict(X_test)

In [ ]:
#a = pd.Series(stack_pred)
#a.index = np.arange(368088,368088+15120,1)
#plot_series(y_train, y_val, a, markers=[',' , ',' , ','])
#plt.show()

In [ ]:
"""submission['공급량'] = pred
submission.to_csv("cat.csv",index=False)"""

In [ ]:
params_lgb = dict()
"""params_lgb['boosting_type'] = ['gbdt', 'dart']
params_lgb['n_jobs'] = [-1]
params_lgb['num_leaves'] = np.arange(50,300)
params_lgb['learning_rate'] = np.linspace(0.0001, 0.5, 1000)
params_lgb['colsample_bytree'] = np.linspace(0.5, 1, 1000)
params_lgb['subsample'] = np.linspace(0.25, 0.9, 1000)
params_lgb['n_estimators'] = np.arange(70, 300)
params_lgb['min_child_weight'] = np.linspace(0.00001, 50, 1000)
params_lgb['min_child_samples'] = np.arange(3, 50)
params_lgb['reg_alpha'] = np.linspace(0.001, 1, 1000)
params_lgb['reg_lambda'] = np.linspace(0.0001, 0.5, 1000)
params_lgb['random_state'] = [21,42,2020,2021]"""
params_lgb['reg_lambda'] = [int(x) for x in np.linspace(start = 10.0, stop = 1e-3, num = 10)]
params_lgb['reg_alpha'] = [int(x) for x in np.linspace(start = 10.0, stop = 1e-3, num = 10)]
params_lgb['colsample_bytree'] = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
params_lgb['subsample'] = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
params_lgb['learning_rate'] = [0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]
params_lgb['n_estimators'] = [int(x) for x in np.linspace(start = 100, stop = 500, num = 100)]
params_lgb['max_depth'] = [int(x) for x in np.linspace(1, 11, num = 1)]
params_lgb['max_depth'].append(None)
params_lgb['random_state'] = [21,42,2020,2021]
params_lgb['cat_smooth'] = [int(x) for x in np.linspace(1,100,10)]
#params_lgb['min_child_weight'] = [int(x) for x in np.linspace(start = 1, stop = 300, num = 10)]

params_xgb = dict()
params_xgb['reg_lambda'] = [int(x) for x in np.linspace(start = 10.0, stop = 1e-3, num = 10)]
params_xgb['reg_alpha'] = [int(x) for x in np.linspace(start = 10.0, stop = 1e-3, num = 10)]
params_xgb['colsample_bytree'] = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
params_xgb['subsample'] = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
params_xgb['learning_rate'] = [0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]
params_xgb['n_estimators'] = [int(x) for x in np.linspace(start = 100, stop = 500, num = 100)]
params_xgb['max_depth'] = [int(x) for x in np.linspace(1, 11, num = 1)]
params_xgb['max_depth'].append(None)
params_xgb['random_state'] = [21,42,2020,2021]
#params_xgb['min_child_weight'] = [int(x) for x in np.linspace(start = 1, stop = 300, num = 10)]

In [ ]:
from sklearn.metrics import make_scorer
score = make_scorer(paul_MAE)

In [ ]:
best = simulate(params, score, 10,
                        LGBMRegressor(),
                        X, y,
                        selection = 'tournament',
                        p_cross = 0.8,
                        p_mutate = 0.5,
                        sim_ann = True)

In [ ]:
print(best)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

def random_rf(data=train):
    train_years = [2013,2014,2015,2016,2017]
    val_years = [2018]
    features = ['구분','시간','month','weekday','quarter']
    x_train = data[data['year'].isin(train_years)] 
    x_val = data[data['year'].isin(val_years)]

    y_train = x_train['공급량']
    y_val = x_val['공급량']

    x_train = x_train[features]
    x_val = x_val[features]
    print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)
    n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 100)]
    max_features = ['auto', 'sqrt']
    max_depth = [int(x) for x in np.linspace(1, 10, num = 1)]
    max_depth.append(None)
    min_samples_split = [2, 5, 10]
    min_samples_leaf = [1, 2, 4]
    random_state = [21,42,2020,2021]
    bootstrap = [True, False]
    rf_params = {
                   'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap,
                   'random_state':random_state,
                }

    rf = RandomForestRegressor()
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = rf_params, n_iter = 100, cv = 3, verbose=2, n_jobs = -1, scoring = 'neg_mean_absolute_error')
    rf_random.fit(x_train, y_train)
    preds = rf_random.predict(x_val)
    paul_mae = paul_MAE(preds, y_val)
    return paul_mae, rf_random.best_params_

def random_xgb(data=train):
    train_years = [2013,2014,2015,2016,2017]
    val_years = [2018]
    features = ['구분','시간','month','weekday','quarter']
    x_train = data[data['year'].isin(train_years)] 
    x_val = data[data['year'].isin(val_years)]

    y_train = x_train['공급량']
    y_val = x_val['공급량']

    x_train = x_train[features]
    x_val = x_val[features]
    print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)
    reg_lambda = [int(x) for x in np.linspace(start = 10.0, stop = 1e-3, num = 10)]
    reg_alpha = [int(x) for x in np.linspace(start = 10.0, stop = 1e-3, num = 10)]
    colsample_bytree = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    subsample = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    learning_rate = [0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]
    n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 100)]
    max_depth = [int(x) for x in np.linspace(1, 11, num = 1)]
    max_depth.append(None)
    random_state = [21,42,2020,2021]
    #min_child_weight = [int(x) for x in np.linspace(start = 1, stop = 300, num = 10)]
    xgb_params = {
                   'lambda':reg_lambda,
                   'alpha':reg_alpha,
                   'colsample_bytree':colsample_bytree,
                   'subsample':subsample,
                   'learning_rate':learning_rate,
                   'n_estimators': n_estimators,
                   'max_depth': max_depth,
                   'random_state':random_state,
                }

    xgb = XGBRegressor()
    xgb_random = RandomizedSearchCV(estimator = xgb, param_distributions = xgb_params, n_iter = 100, cv = 3, verbose=2, n_jobs = -1, scoring = 'neg_mean_absolute_error')
    xgb_random.fit(x_train, y_train)
    preds = xgb_random.predict(x_val)
    paul_mae = paul_MAE(preds, y_val)
    return paul_mae, xgb_random.best_params_

def random_lgb(data=train):
    train_years = [2013,2014,2015,2016,2017]
    val_years = [2018]
    features = ['구분','시간','month','weekday','quarter']
    x_train = data[data['year'].isin(train_years)] 
    x_val = data[data['year'].isin(val_years)]

    y_train = x_train['공급량']
    y_val = x_val['공급량']

    x_train = x_train[features]
    x_val = x_val[features]
    print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)    
    reg_lambda = [int(x) for x in np.linspace(start = 10.0, stop = 1e-3, num = 10)]
    reg_alpha = [int(x) for x in np.linspace(start = 10.0, stop = 1e-3, num = 10)]
    colsample_bytree = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    subsample = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    learning_rate = [0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]
    n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 100)]
    max_depth = [int(x) for x in np.linspace(1, 11, num = 1)]
    max_depth.append(None)
    random_state = [21,42,2020,2021]
    cat_smooth = [int(x) for x in np.linspace(1,100,10)]
    #min_child_weight = [int(x) for x in np.linspace(start = 1, stop = 300, num = 10)]
    lgb_params = {
                   'lambda':reg_lambda,
                   'alpha':reg_alpha,
                   'colsample_bytree':colsample_bytree,
                   'subsample':subsample,
                   'learning_rate':learning_rate,
                   'n_estimators': n_estimators,
                   'max_depth': max_depth,
                   'random_state':random_state,
                   'cat_smooth':cat_smooth,
                }

    lgb = LGBMRegressor()
    lgb_random = RandomizedSearchCV(estimator = lgb, param_distributions = lgb_params, n_iter = 100, cv = 3, verbose=2, n_jobs = -1, scoring = 'neg_mean_absolute_error')
    lgb_random.fit(x_train, y_train)
    preds = lgb_random.predict(x_val)
    paul_mae = paul_MAE(preds, y_val)
    return paul_mae, lgb_random.best_params_

def random_cat(data=train):
    train_years = [2013,2014,2015,2016,2017]
    val_years = [2018]
    features = ['구분','시간','month','weekday','quarter']
    x_train = data[data['year'].isin(train_years)] 
    x_val = data[data['year'].isin(val_years)]

    y_train = x_train['공급량']
    y_val = x_val['공급량']

    x_train = x_train[features]
    x_val = x_val[features]
    print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)    
                
    od_wait = [int(x) for x in np.linspace(start = 500, stop=1000, num = 100)]
    iterations = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 100)]
    reg_lambda = [int(x) for x in np.linspace(start = 10.0, stop = 1e-3, num = 10)]
    subsample = [0, 0.1, 0.2 ,0.3 ,0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    learning_rate = [0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]
    max_depth = [int(x) for x in np.linspace(1, 10, num = 1)]
    max_depth.append(None)
    random_strength = [21,42,2020,2021]
    min_data_in_leaf = [int(x) for x in np.linspace(start = 1, stop = 30, num = 1)]
    leaf_estimation_iterations = [int(x) for x in np.linspace(start = 1, stop = 15, num = 1)]
    #min_child_weight = [int(x) for x in np.linspace(start = 1, stop = 300, num = 10)]
    cat_params = {
                   'leaf_estimation_method':['Gradient', 'Newton', 'Exact', 'Simple'],
                   'bootstrap_type': ['Poisson', 'Bayesian', 'Bernoulli', 'MVS', 'No'],
                   'reg_lambda':reg_lambda,
                   'subsample':subsample,
                   'iterations':iterations,
                   'learning_rate':learning_rate,
                   'max_depth': max_depth,
                   'random_strength':random_strength,
                   #'min_data_in_leaf':min_data_in_leaf,
                   #'leaf_estimation_iterations':leaf_estimation_iterations,
                   'od_wait':od_wait,
                }

    cat = CatBoostRegressor()
    cat_random = RandomizedSearchCV(estimator = cat, param_distributions = cat_params, n_iter = 100, cv = 3, verbose=2, n_jobs = -1, scoring = 'neg_mean_absolute_error')
    cat_random.fit(x_train, y_train)
    preds = cat_random.predict(x_val)
    paul_mae = paul_MAE(preds, y_val)
    return paul_mae, cat_random.best_params_

In [ ]:
rf_paul_mae, rf_best = random_rf()
rf_paul_mae, rf_best

In [ ]:
xgb_paul_mae, xgb_best = random_xgb()
xgb_paul_mae, xgb_best

In [ ]:
lgb_paul_mae, lgb_best = random_lgb()
lgb_paul_mae, lgb_best

In [ ]:
cat_paul_mae, cat_best = random_cat()
cat_paul_mae, cat_best

In [ ]:
def objective_xgb(trial, data=train):
    train_years = [2013,2014,2015,2016,2017]
    val_years = [2018]
    features = ['구분','시간','month','weekday','quarter']
    x_train = data[data['year'].isin(train_years)] 
    x_val = data[data['year'].isin(val_years)]

    y_train = x_train['공급량']
    y_val = x_val['공급량']

    x_train = x_train[features]
    x_val = x_val[features]
    print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)
    
    params = {
        #'lambda':trial.suggest_loguniform('lambda',1e-3, 10.0),
        #'alpha':trial.suggest_loguniform('alpha',1e-3, 10.0),
        'verbose': -1,
        'metric': 'mae', 
        'colsample_bytree':trial.suggest_categorical('colsmaple_bytree',[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample':trial.suggest_categorical('subsample',[0.1, 0.2 ,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'learning_rate':trial.suggest_categorical('learning_rate',[0.2, 0.15, 0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]),
        'n_estimators':trial.suggest_categorical('n_estimators',[100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]),
        #'max_depth':trial.suggest_categorical('max_depth',[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, None]),
        'random_state':trial.suggest_categorical('random_state',[11,21,42,2020,2021]),
        #'min_child_weight':trial.suggest_int('min_child_weight',1,300),
    }
    model = XGBRegressor(**params)
    model.fit(x_train,y_train,verbose=False)
    preds = model.predict(x_val)
    paul_mae = paul_MAE(preds, y_val)
    return paul_mae

def objective_lgb(trial, data=train):
    train_years = [2013,2014,2015,2016,2017]
    val_years = [2018]
    features = ['구분','시간','month','weekday','quarter']
    x_train = data[data['year'].isin(train_years)] 
    x_val = data[data['year'].isin(val_years)]

    y_train = x_train['공급량']
    y_val = x_val['공급량']

    x_train = x_train[features]
    x_val = x_val[features]
    print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)
    
    params = {
        'objective':"regression",
        'verbose': -1,
        'metric': 'mae', 
        'n_estimators':trial.suggest_categorical('n_estimators',[100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]),
        'random_state':trial.suggest_categorical('random_state',[11, 21, 42, 2020, 2021]),
        #'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        #'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.1, 0.2 ,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.2, 0.15, 0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]),
        #'max_depth': trial.suggest_categorical('max_depth', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, None]),
        #'num_leaves' : trial.suggest_int('num_leaves', 1, 100),
        'feature_fraction':trial.suggest_uniform('feature_fraction',0.1,1),
        'bagging_fraction':trial.suggest_uniform('bagging_fraction',0.1,1),
        'bagging_freq':trial.suggest_int('bagging_freq',1,10),
        #'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        #'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    model = LGBMRegressor(**params)
    model.fit(x_train,y_train,verbose=False)
    preds = model.predict(x_val)
    paul_mae = paul_MAE(preds, y_val)
    return paul_mae

def objective_cat(trial, data=train):
    train_years = [2013,2014,2015,2016,2017]
    val_years = [2018]
    features = ['구분','시간','month','weekday','quarter']
    x_train = data[data['year'].isin(train_years)] 
    x_val = data[data['year'].isin(val_years)]

    y_train = x_train['공급량']
    y_val = x_val['공급량']

    x_train = x_train[features]
    x_val = x_val[features]
    print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)
    
    params = {
            'iterations':trial.suggest_categorical('iterations',[100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]),
            #'od_wait':trial.suggest_int('od_wait', 100, 500),
            #'leaf_estimation_method':trial.suggest_categorical('leaf_estimation_method',['Gradient', 'Newton', 'Exact', 'Simple']),
            #'bootstrap_type': trial.suggest_categorical('bootstrap_type',['Poisson', 'Bayesian', 'Bernoulli', 'MVS', 'No']),
            'bootstrap_type':'Bernoulli',
            'learning_rate': trial.suggest_categorical('learning_rate', [0.2, 0.15, 0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]),
            #'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
            'subsample': trial.suggest_categorical('subsample', [0.1, 0.2 ,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
            'random_strength':trial.suggest_categorical('random_strength',[11, 21, 42, 2020, 2021]),
            #'depth': trial.suggest_categorical('depth', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, None]),
            #'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
            #'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15)
    }
    model = CatBoostRegressor(**params)
    model.fit(x_train,y_train,verbose=False)
    preds = model.predict(x_val)
    paul_mae = paul_MAE(preds, y_val)
    return paul_mae

In [ ]:
from optuna.samplers import TPESampler

In [ ]:
from optuna.samplers import TPESampler
sampler = TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective_xgb, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
xgb_best = study.best_trial.params

In [ ]:
study_xgb = study.trials_dataframe()
study_xgb_ = study_xgb[study_xgb.value > 0]
study_xgb_.sort_values(by = 'value', ascending = False)

In [ ]:
sampler = TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective_lgb, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
lgb_best = study.best_trial.params

In [ ]:
#xgb_best = {'colsmaple_bytree': 0.2, 'subsample': 0.2, 'learning_rate': 0.07, 'n_estimators': 300, 'max_depth': 9, 'random_state': 21}
xgb_best ={'colsmaple_bytree': 0.3, 'subsample': 0.6, 'learning_rate': 0.09, 'n_estimators': 600, 'random_state': 21}
#lgb_best = {'n_estimators': 1000, 'random_state': 11, 'colsample_bytree': 0.9, 'subsample': 0.6, 'learning_rate': 0.15, 'max_depth': 4, 'feature_fraction': 0.47013836621824984, 'bagging_fraction': 0.5465284168641787, 'bagging_freq': 4}
lgb_best = {'n_estimators': 200, 'random_state': 2021, 'colsample_bytree': 0.1, 'subsample': 1.0, 'learning_rate': 0.05, 'feature_fraction': 0.20891622941413984, 'bagging_fraction': 0.5482767726996648, 'bagging_freq': 8}
#cat_best = {'iterations': 400, 'learning_rate': 0.07, 'subsample': 0.5, 'random_strength': 2020, 'depth': None}
cat_best = {'iterations': 300, 'learning_rate': 0.08, 'subsample': 0.9, 'random_strength': 21}

In [ ]:
study_lgb = study.trials_dataframe()
study_lgb_ = study_lgb[study_lgb.value > 0]
study_lgb_.sort_values(by = 'value', ascending = False)

In [ ]:
sampler = TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective_cat, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
cat_best = study.best_trial.params

In [ ]:
study_cat = study.trials_dataframe()
study_cat_ = study_cat[study_cat.value > 0]
study_cat_.sort_values(by = 'value', ascending = False)

In [ ]:
print(study_cat_.sort_values(by = 'value', ascending = False).iloc[0])
#{'depth'=10,'iterations'=2000,'leaf_estimation_iterations'=10,'learning_rate'=0.608965,'min_data_in_leaf'=20,'od_wait'=1559,'random_strength'=38.042741,'reg_lambda'=98.344238,'subsample'=0.998485}
print(study_lgb_.sort_values(by = 'value', ascending = False).iloc[0])
#{'colsample_bytree'=1.0,'learning_rate'=0.014,'max_depth'=3,'min_child_samples'=33,'min_data_per_groups'=32,'n_estimators'=1000,'num_leaves'=328,'random_state'=2021,'reg_alpha'=0.093796,'reg_lambda'=1.239346,'subsample'=0.6}
print(study_xgb_.sort_values(by = 'value', ascending = False).iloc[0])
#{'alpha'=0.878075,'colsmaple_bytree'=0.3,'lambda'=0.004216,'learning_rate'=0.01,'max_depth'=7,'min_child_weight'=272,'n_estimators'=1000,'random_state'=2020,'subsample'=0.8}

In [ ]:
"""
import lightgbm as lgb

d_train = lgb.Dataset(x_train, y_train)
d_val = lgb.Dataset(x_val, y_val)

params = {
    'boosting_type':'gbdt',
    'metric':"mae",
    'random_state':42
}

model_lgb = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=100)
preds_lgb = model_lgb.predict(X_test)
submission['공급량'] = preds_lgb
submission.to_csv('LGB.csv', index=False)
"""

In [ ]:
"""
import xgboost as xgb

dtrain = xgb.DMatrix(data=x_train, label = y_train)
dval = xgb.DMatrix(data=x_val, label = y_val)
wlist = [(dtrain, 'train'), (dval,'eval')]

params = {
    'learning_rate': 0.05,
    'metric':'mae', 
    'seed':42
}
 

model_xgb = xgb.train( params, dtrain, 500, evals=wlist, verbose_eval=20, early_stopping_rounds=100)
test_x = xgb.DMatrix(X_test)
preds_xgb = model_xgb.predict(test_x)
submission['공급량'] = preds_xgb
submission.to_csv('XGB.csv', index=False)
"""

In [ ]:
"""
model = CatBoostRegressor(iterations=650, learning_rate=0.03)
eval_dataset = Pool(x_val,y_val)
model.fit(x_train,y_train,eval_set=eval_dataset,
          verbose=False)
pred = model.predict(X_test)
submission['공급량'] = pred
submission.to_csv('CAT.csv', index=False)
"""

In [ ]:
#!pip install tslearn

In [ ]:
"""from tslearn.clustering import TimeSeriesKMeans
kmean = TimeSeriesKMeans(n_clusters=7, metric="dtw").fit(X) # "euclidean" softdtw
#gmm = GaussianMixture(n_components=6, random_state=42, n_init=20).fit(X)
#kmean = KMeans(n_clusters=10, random_state=42).fit(X)
X_kmean_labels = kmean.predict(X)
X_test_kmean_labels = kmean.predict(X_test)

X['kmean_cluster'] = X_kmean_labels
X_test['kmean_cluster'] = X_test_kmean_labels"""

In [ ]:
#y_array = np.array(y).reshape(-1,1)
#pt = PowerTransformer(method='yeo-johnson') #method='yeo-johnson', standardize=True
#y_array = pt.fit_transform(y_array)

In [ ]:
"""def valid_eval_plot(models):
    m_clf_preds = []
    m_clf_score = []
    for m in models:
        m_clf = m
        m_clf.fit(x_train,y_train)
        m_clf_pred = m_clf.predict(x_val)
        m_clf_preds.append(m_clf_pred)
        m_clf_score.append(paul_MAE(y_val,m_clf_pred))
    for i in range(1,len(models)+1):
        plt.subplot(len(models)+1,1,i)
        pred = pd.Series(m_clf_preds[i-1])
        pred.index = np.arange(y_val.index[0], y_val.index[-1]+1)
        plot_series(y_train, y_val, pd.Series(pred), markers = [',' , ',' , ','])
        plt.title(str(models[i-1]) + ' / mae: ' + str(m_clf_score[i-1]))
        plt.xticks([])
        plt.yticks([])
        plt.show()

# kernel = DotProduct() + WhiteKernel()
# GaussianProcessRegressor(kernel=kernel)
# 스태킹X -> Lars(), RidgeCV(), Ridge(), ElasticNet, LinearRegression, OrthogonalMatchingPursuit(), GaussianProcessRegressor(), DummyRegressor(), KernelRidge(), SVC(), GaussianNB(), LassoLars()
# 스태킹O -> KNeighborsRegressor(), BaggingRegressor(), ExtraTreesRegressor(), CatBoostRegressor(), LGBMRegressor(), GradientBoostingRegressor(), RandomForestRegressor(), DecisionTreeRegressor(), ExtraTreeRegressor(), MLPRegressor()
model_list = [XGBRegressor(), KNeighborsRegressor(), BaggingRegressor(), ExtraTreesRegressor(), LGBMRegressor(), GradientBoostingRegressor(), RandomForestRegressor(), DecisionTreeRegressor(), ExtraTreeRegressor(), MLPRegressor()]
valid_eval_plot(model_list)"""

https://dacon.io/codeshare/2366?dtype=recent
https://www.kaggle.com/artgor/quick-and-dirty-regression
https://towardsdatascience.com/ngboost-explained-comparison-to-lightgbm-and-xgboost-fda510903e53

In [ ]:
#{'alpha'=0.878075,'colsmaple_bytree'=0.3,'lambda'=0.004216,'learning_rate'=0.01,'max_depth'=7,'min_child_weight'=272,'n_estimators'=1000,'random_state'=2020,'subsample'=0.8}
#{'colsample_bytree'=1.0,'learning_rate'=0.014,'max_depth'=3,'min_child_samples'=33,'min_data_per_groups'=32,'n_estimators'=1000,'num_leaves'=328,'random_state'=2021,'reg_alpha'=0.093796,'reg_lambda'=1.239346,'subsample'=0.6}
#{'depth'=10,'iterations'=2000,'leaf_estimation_iterations'=10,'learning_rate'=0.608965,'min_data_in_leaf'=20,'od_wait'=1559,'random_strength'=38.042741,'reg_lambda'=98.344238,'subsample'=0.998485}

In [ ]:
#rf_bset

In [ ]:
#model = LGBMRegressor(reg_alpha=0.878075,colsample_bytree=0.3,reg_lambda=0.004216,learning_rate=0.01,max_depth=7,min_child_weight=272,n_estimators=1000,random_state=2020,subsample=0.8)
#model = CatBoostRegressor(depth=10,iterations=2000,leaf_estimation_iterations=10,learning_rate=0.608965,min_data_in_leaf=20,od_wait=1559,random_strength=38.042741,reg_lambda=98.344238,subsample=0.998485)
#model = XGBRegressor(alpha = 0.01643163148521856, colsmaple_bytree = 0.3, subsample = 0.4, learning_rate = 0.03, n_estimators = 500, max_depth = 11, random_state = 42, min_child_weight = 181, reg_lambda = 0.4497515909263729,)
model = XGBRegressor(learning_rate = 0.1, metric = MAPE, random_state = 42)
model.fit(x_train,y_train)
pred = model.predict(x_val)
pred = pd.Series(pred)
pred.index = np.arange(y_val.index[0], y_val.index[-1]+1)
plot_series(y_train, y_val, pd.Series(pred), markers=[',' , ',', ','])
plt.title(str(model) + ' / mae: ' + str(paul_MAE(y_val,pred)))

#HistGradientBoostingRegressor() / mae: 0.518471462402873'
#LGBMRegressor(boosting_type='gbdt',objective="regression",metric="mae",random_state=42) / mae: 0.5331642457461008"
#CatBoostRegressor(learning_rate = 0.1, bootstrap_type = 'Bernoulli') / mae: 0.5520655168232055'
#BaggingRegressor() / mae: 0.5651288906591797'
#ExtraTreesRegressor() / mae: 0.5647851034364201'
#DecisionTreeRegressor() / mae: 0.5647851432512444'
#ExtraTreeRegressor() / mae: 0.5647851432512443'
#XGBRegressor(learning_rate = 0.1, metric = MAE, random_state = 42) / mae: 0.5418492985417777"
#RandomForestRegressor() / mae: 0.5648324187813311'


#MultiOutputRegressor
#VotingRegressor 

In [ ]:
#model = LGBMRegressor(reg_alpha=0.878075,colsample_bytree=0.3,reg_lambda=0.004216,learning_rate=0.01,max_depth=7,min_child_weight=272,n_estimators=1000,random_state=2020,subsample=0.8)
#model = CatBoostRegressor(depth=10,iterations=2000,leaf_estimation_iterations=10,learning_rate=0.608965,min_data_in_leaf=20,od_wait=1559,random_strength=38.042741,reg_lambda=98.344238,subsample=0.998485)
#model = XGBRegressor(alpha = 0.01643163148521856, colsmaple_bytree = 0.3, subsample = 0.4, learning_rate = 0.03, n_estimators = 500, max_depth = 11, random_state = 42, min_child_weight = 181, reg_lambda = 0.4497515909263729,)
model = LGBMRegressor(metric = MAPE)
model.fit(x_train,y_train)
pred = model.predict(x_val)
pred = pd.Series(pred)
pred.index = np.arange(y_val.index[0], y_val.index[-1]+1)
plot_series(y_train, y_val, pd.Series(pred), markers=[',' , ',', ','])
plt.title(str(model) + ' / mae: ' + str(paul_MAE(y_val,pred)))

#HistGradientBoostingRegressor() / mae: 0.518471462402873'
#LGBMRegressor(boosting_type='gbdt',objective="regression",metric="mae",random_state=42) / mae: 0.5331642457461008"
#CatBoostRegressor(learning_rate = 0.1, bootstrap_type = 'Bernoulli') / mae: 0.5520655168232055'
#BaggingRegressor() / mae: 0.5651288906591797'
#ExtraTreesRegressor() / mae: 0.5647851034364201'
#DecisionTreeRegressor() / mae: 0.5647851432512444'
#ExtraTreeRegressor() / mae: 0.5647851432512443'
#XGBRegressor(learning_rate = 0.1, metric = MAE, random_state = 42) / mae: 0.5418492985417777"
#RandomForestRegressor() / mae: 0.5648324187813311'


#MultiOutputRegressor
#VotingRegressor

In [ ]:
#model = LGBMRegressor(reg_alpha=0.878075,colsample_bytree=0.3,reg_lambda=0.004216,learning_rate=0.01,max_depth=7,min_child_weight=272,n_estimators=1000,random_state=2020,subsample=0.8)
#model = CatBoostRegressor(depth=10,iterations=2000,leaf_estimation_iterations=10,learning_rate=0.608965,min_data_in_leaf=20,od_wait=1559,random_strength=38.042741,reg_lambda=98.344238,subsample=0.998485)
#model = XGBRegressor(alpha = 0.01643163148521856, colsmaple_bytree = 0.3, subsample = 0.4, learning_rate = 0.03, n_estimators = 500, max_depth = 11, random_state = 42, min_child_weight = 181, reg_lambda = 0.4497515909263729,)
model = HistGradientBoostingRegressor(metric = MAPE)
model.fit(x_train,y_train)
pred = model.predict(x_val)
pred = pd.Series(pred)
pred.index = np.arange(y_val.index[0], y_val.index[-1]+1)
plot_series(y_train, y_val, pd.Series(pred), markers=[',' , ',', ','])
plt.title(str(model) + ' / mae: ' + str(paul_MAE(y_val,pred)))

#HistGradientBoostingRegressor() / mae: 0.518471462402873'
#LGBMRegressor(boosting_type='gbdt',objective="regression",metric="mae",random_state=42) / mae: 0.5331642457461008"
#CatBoostRegressor(learning_rate = 0.1, bootstrap_type = 'Bernoulli') / mae: 0.5520655168232055'
#BaggingRegressor() / mae: 0.5651288906591797'
#ExtraTreesRegressor() / mae: 0.5647851034364201'
#DecisionTreeRegressor() / mae: 0.5647851432512444'
#ExtraTreeRegressor() / mae: 0.5647851432512443'
#XGBRegressor(learning_rate = 0.1, metric = MAE, random_state = 42) / mae: 0.5418492985417777"
#RandomForestRegressor() / mae: 0.5648324187813311'


#MultiOutputRegressor
#VotingRegressor

In [ ]:
#model = LGBMRegressor(reg_alpha=0.878075,colsample_bytree=0.3,reg_lambda=0.004216,learning_rate=0.01,max_depth=7,min_child_weight=272,n_estimators=1000,random_state=2020,subsample=0.8)
#model = CatBoostRegressor(depth=10,iterations=2000,leaf_estimation_iterations=10,learning_rate=0.608965,min_data_in_leaf=20,od_wait=1559,random_strength=38.042741,reg_lambda=98.344238,subsample=0.998485)
#model = XGBRegressor(alpha = 0.01643163148521856, colsmaple_bytree = 0.3, subsample = 0.4, learning_rate = 0.03, n_estimators = 500, max_depth = 11, random_state = 42, min_child_weight = 181, reg_lambda = 0.4497515909263729,)
model = CatBoostRegressor(learning_rate = 0.1, bootstrap_type = 'Bernoulli', metric = MAPE)
model.fit(x_train,y_train)
pred = model.predict(x_val)
pred = pd.Series(pred)
pred.index = np.arange(y_val.index[0], y_val.index[-1]+1)
plot_series(y_train, y_val, pd.Series(pred), markers=[',' , ',', ','])
plt.title(str(model) + ' / mae: ' + str(paul_MAE(y_val,pred)))

#HistGradientBoostingRegressor() / mae: 0.518471462402873'
#LGBMRegressor(boosting_type='gbdt',objective="regression",metric="mae",random_state=42) / mae: 0.5331642457461008"
#CatBoostRegressor(learning_rate = 0.1, bootstrap_type = 'Bernoulli') / mae: 0.5520655168232055'
#BaggingRegressor() / mae: 0.5651288906591797'
#ExtraTreesRegressor() / mae: 0.5647851034364201'
#DecisionTreeRegressor() / mae: 0.5647851432512444'
#ExtraTreeRegressor() / mae: 0.5647851432512443'
#XGBRegressor(learning_rate = 0.1, metric = MAE, random_state = 42) / mae: 0.5418492985417777"
#RandomForestRegressor() / mae: 0.5648324187813311'


#MultiOutputRegressor
#VotingRegressor

In [ ]:
"""tabnet_params = dict(
    n_d=24,
    n_a=24,
    n_steps=1,
    gamma=1.3,
    lambda_sparse=0,
    optimizer_fn = optim.SGD,
    optimizer_params = dict(lr = 1e-2, weight_decay=1e-5),
    mask_type = "entmax", #sparsemax, entmax
    scheduler_params = dict(
        mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
    scheduler_fn = ReduceLROnPlateau,
    seed = 2021,
    verbose = 10,
)"""

In [ ]:
#model = LGBMRegressor(reg_alpha=0.878075,colsample_bytree=0.3,reg_lambda=0.004216,learning_rate=0.01,max_depth=7,min_child_weight=272,n_estimators=1000,random_state=2020,subsample=0.8)
#model = CatBoostRegressor(depth=10,iterations=2000,leaf_estimation_iterations=10,learning_rate=0.608965,min_data_in_leaf=20,od_wait=1559,random_strength=38.042741,reg_lambda=98.344238,subsample=0.998485)
#model = XGBRegressor(alpha = 0.01643163148521856, colsmaple_bytree = 0.3, subsample = 0.4, learning_rate = 0.03, n_estimators = 500, max_depth = 11, random_state = 42, min_child_weight = 181, reg_lambda = 0.4497515909263729,)
#x_train_v = x_train.values
#y_train_v = y_train.values.reshape(-1,1)
#x_val_v = x_val.values
#y_val_v = y_val.values.reshape(-1,1)
#model = TabNetRegressor(**tabnet_params)
#model.fit(x_train_v, y_train_v, eval_set=[(x_val_v, y_val_v)], eval_metric=['mae'], eval_name=['val'], max_epochs=20, patience=10, batch_size=128) #, patience=10, batch_size=1024*2, virtual_batch_size=128*2,
#pred = model.predict(x_val_v)
#pred = pd.Series(pred)
#pred.index = np.arange(y_val.index[0], y_val.index[-1]+1)
#plot_series(y_train, y_val, pd.Series(pred), markers=[',' , ',', ','])
#plt.title(str(model) + ' / mae: ' + str(paul_MAE(y_val,pred)))

#HistGradientBoostingRegressor() / mae: 0.518471462402873'
#LGBMRegressor(boosting_type='gbdt',objective="regression",metric="mae",random_state=42) / mae: 0.5331642457461008"
#CatBoostRegressor(learning_rate = 0.1, bootstrap_type = 'Bernoulli') / mae: 0.5520655168232055'
#BaggingRegressor() / mae: 0.5651288906591797'
#ExtraTreesRegressor() / mae: 0.5647851034364201'
#DecisionTreeRegressor() / mae: 0.5647851432512444'
#ExtraTreeRegressor() / mae: 0.5647851432512443'
#XGBRegressor(learning_rate = 0.1, metric = MAE, random_state = 42) / mae: 0.5418492985417777"
#RandomForestRegressor() / mae: 0.5648324187813311'


#MultiOutputRegressor
#VotingRegressor

In [ ]:
model = TabNetRegressor(gamma=1.1, momentum=0.02, seed=42)
model.fit(x_train_v, y_train_v, eval_set=[(x_val_v, y_val_v)], eval_metric=['mae'], eval_name=['val'], max_epochs=20) #, patience=10, batch_size=1024*2, virtual_batch_size=128*2,

In [ ]:
X_v = X.values
y_log_v = y_log.values.reshape(-1,1)
X_test_v = X_test.values

In [ ]:
tabnet_params = dict(
    n_d = 32,
    n_a = 32,
    n_steps = 3,
    gamma = 1.3,
    lambda_sparse = 0,
    optimizer_fn = optim.Adam,
    optimizer_params = dict(lr = 1e-2, weight_decay = 1e-5),
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
    scheduler_fn = ReduceLROnPlateau,
    seed = 42,
)

In [ ]:
tabnet = TabNetRegressor(**tabnet_params)
tabnet.fit(X_v, y_log_v, max_epochs=20)
pred = tabnet.predict(X_test_v)
pred

In [ ]:
import itertools
pred_list = list(itertools.chain(*pred))
pred_all = []
for i in pred_list:
    pred_all.append(np.round(i,3))
pred_all = np.array(pred_all)

In [ ]:
pred_all_exp = np.expm1(pred_all)
pred_all_exp

In [ ]:
submission['공급량'] = np.round(pred_all_exp,3)
submission.to_csv('[D-13]TabnetRegressor.csv',index=False)
submission

In [ ]:
#model = LGBMRegressor(reg_alpha=0.878075,colsample_bytree=0.3,reg_lambda=0.004216,learning_rate=0.01,max_depth=7,min_child_weight=272,n_estimators=1000,random_state=2020,subsample=0.8)
#model = CatBoostRegressor(depth=10,iterations=2000,leaf_estimation_iterations=10,learning_rate=0.608965,min_data_in_leaf=20,od_wait=1559,random_strength=38.042741,reg_lambda=98.344238,subsample=0.998485)
#model = XGBRegressor(alpha = 0.01643163148521856, colsmaple_bytree = 0.3, subsample = 0.4, learning_rate = 0.03, n_estimators = 500, max_depth = 11, random_state = 42, min_child_weight = 181, reg_lambda = 0.4497515909263729,)
x_train_v = x_train.values
y_train_v = y_train.values.reshape(-1,1)
x_val_v = x_val.values
y_val_v = y_val.values.reshape(-1,1)
model = TabNetRegressor(gamma=1.1, momentum=0.02, seed=42)
model.fit(x_train_v, y_train_v, eval_set=[(x_val_v, y_val_v)], eval_metric=['mae'], eval_name=['val'], max_epochs=20) #, patience=10, batch_size=1024*2, virtual_batch_size=128*2,
pred = model.predict(x_val_v)
pred = pred.reshape(61320,)
pred = pd.Series(pred)
pred.index = np.arange(y_val.index[0], y_val.index[-1]+1)
plot_series(y_train, y_val, pd.Series(pred), markers=[',' , ',', ','])
plt.title(str(model) + ' / mae: ' + str(paul_MAE(y_val,pred)))

#HistGradientBoostingRegressor() / mae: 0.518471462402873'
#LGBMRegressor(boosting_type='gbdt',objective="regression",metric="mae",random_state=42) / mae: 0.5331642457461008"
#CatBoostRegressor(learning_rate = 0.1, bootstrap_type = 'Bernoulli') / mae: 0.5520655168232055'
#BaggingRegressor() / mae: 0.5651288906591797'
#ExtraTreesRegressor() / mae: 0.5647851034364201'
#DecisionTreeRegressor() / mae: 0.5647851432512444'
#ExtraTreeRegressor() / mae: 0.5647851432512443'
#XGBRegressor(learning_rate = 0.1, metric = MAE, random_state = 42) / mae: 0.5418492985417777"
#RandomForestRegressor() / mae: 0.5648324187813311'


#MultiOutputRegressor
#VotingRegressor

# DeepLearningModel

In [ ]:
import random
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, TimeDistributed
from tensorflow.python.keras.layers import Dense, LSTM, Dropout,Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import mean_absolute_percentage_error

"""import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters"""

In [ ]:
x_train = x_train.values
x_train_t = x_train.reshape(x_train.shape[0],6,1)
x_val = x_val.values
x_val_t = x_val.reshape(x_val.shape[0],6,1)
y_train_t = y_train.values
y_val_t = y_val.values

X = X.values
X_t = X.reshape(X.shape[0],6,1)
X_test_t = X_test.values
X_test_t = X_test_t.reshape(X_test.shape[0],6,1)


y_t = y.values
#y_log_t = y_log.values

In [ ]:
optimizer_lst = ['Adam','Nadam','RMSprop']

def CNN(X_test_t,X_t,y_t):
    model = Sequential()
    model.add(Conv1D(filters = 64, kernel_size=4, activation = 'relu', input_shape = (6,1))) # filters = 32, kernel_size=4, 64,2
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam',loss='mean_absolute_percentage_error')
    model.fit(X_t,y_t, epochs=20, verbose=1) # epochs = 300, 600
    pred = model.predict(X_test_t)
    return pred, model

"""def LSTM(X_test_t,X_t,y_t):
    inputs = keras.layers.Input(shape=(5,1))
    lstm_out = keras.layers.LSTM(20)(inputs)
    outputs = keras.layers.Dense(1)(lstm_out)
    #lstm_1_out = keras.layers.LSTM(128, return_sequences=True)(inputs)
    #lstm_2_out = keras.layers.LSTM(64, return_sequences=True)(lstm_1_out)
    #lstm_3_out = keras.layers.LSTM(10, activation='relu')(lstm_2_out)
    #outputs = keras.layers.Dense(1)(lstm_3_out)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mean_absolute_percentage_error')
    #model = Sequential()
    #model.add(keras.layers.LSTM(128, return_sequences=True))
    #model.add(Dense(1))
    model.compile(optimizer='adam',loss='mean_absolute_percentage_error')
    model.fit(X_t,y_t, epochs=20, verbose=1)
    pred = model.predict(X_test_t)
    return pred, model"""

def LSTM(X_test_t, X_t, y_t):
    inputs = keras.layers.Input(shape=(6,1))
    lstm_out1 = keras.layers.LSTM(50,  return_sequences=True)(inputs)
    lstm_out2 = keras.layers.Dropout(0.2)(lstm_out1)
    lstm_out3 = keras.layers.LSTM(100,  return_sequences=False)(lstm_out2)
    lstm_out4 = keras.layers.Dropout(0.2)(lstm_out3)
    outputs = keras.layers.Dense(1, activation='elu')(lstm_out4)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mean_absolute_percentage_error')
    model.fit(X_t, y_t, epochs=20, verbose=1)
    pred = model.predict(X_test_t)
    return pred, model
    

def DNN(X_test_t,X_t,y_t):
    model = Sequential()
    model.add(Dense(units=16, activation='relu', input_shape=(6,)))
    model.add(Dense(units=8, activation='relu'))
    model.add(Dense(units=4, activation='relu'))
    model.add(Dense(units=1))
    model.compile(optimizer='adam',loss='mean_absolute_percentage_error')
    model.fit(X_t,y_t, epochs=20, verbose=1)
    pred = model.predict(X_test_t)
    return pred, model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Flatten,MaxPooling1D,BatchNormalization, Lambda, AveragePooling1D, Dropout, Input

# dacon two bigs team model (6th)
def set_model(X_test_t,X_t,y_t):
    activation = 'elu'
    padding = 'causal'
    nf = 16 
    fs = 3 
    model = Sequential()
    model.add(keras.layers.InputLayer((6, 1)))
    model.add(Conv1D(filters = nf, kernel_size = fs, padding = padding))
    model.add(BatchNormalization()) # 비선형 성질을 유지 하면서 학습 될 수 있게 해주고, regularization 효과를 가지기 위해 배치정규화 설정
    model.add(Activation(activation = activation))
    # model.add(Dropout(0.4))
    
    model.add(Conv1D(filters = nf * 2, kernel_size = fs, padding = padding))
    model.add(BatchNormalization()) # 비선형 성질을 유지 하면서 학습 될 수 있게 해주고, regularization 효과를 가지기 위해 배치정규화 설정
    model.add(Activation(activation = activation))
    # model.add(Dropout(0.4))
    
    model.add(Conv1D(filters = nf * 4, kernel_size = fs, padding = padding))
    model.add(BatchNormalization()) # 비선형 성질을 유지 하면서 학습 될 수 있게 해주고, regularization 효과를 가지기 위해 배치정규화 설정
    model.add(Activation(activation = activation))
    # model.add(Dropout(0.4))
    
    model.add(Conv1D(filters = nf * 8, kernel_size = fs, padding = padding))
    model.add(BatchNormalization()) # 비선형 성질을 유지 하면서 학습 될 수 있게 해주고, regularization 효과를 가지기 위해 배치정규화 설정
    model.add(Activation(activation = activation))
    # model.add(Dropout(0.4))
    
    model.add(Conv1D(filters = nf * 16, kernel_size = fs, padding = padding))
    model.add(BatchNormalization()) # 비선형 성질을 유지 하면서 학습 될 수 있게 해주고, regularization 효과를 가지기 위해 배치정규화 설정
    model.add(Activation(activation = activation))
    # model.add(Dropout(0.4))
    
    model.add(Conv1D(filters = nf * 32, kernel_size = fs, padding = padding))
    model.add(BatchNormalization()) # 비선형 성질을 유지 하면서 학습 될 수 있게 해주고, regularization 효과를 가지기 위해 배치정규화 설정
    model.add(Activation(activation = activation))
    # model.add(Dropout(0.4))
    
    # Convolution 연산을 진행한 결과를 다시 Flatten 하여 최종 값(Dense(1)) 도출
    model.add(Flatten())
    model.add(Dense(512, activation = activation))
    model.add(Dense(128, activation = activation)) 
    model.add(Dense(32, activation = activation)) 
    model.add(Dense(8, activation = activation)) 
    model.add(Dense(4, activation = activation)) 
    model.add(Dense(1))# output size
    
    # Optimization의 경우 RMSprop으로 선택(시계열의 경우 RMSprop이 조금 더 우수한 성능을 보임)
    optimizer = keras.optimizers.RMSprop() # 실제로 RMSprop, SGD, Adam 등을 비교해본 결과 RMSprop가 가장 우수했음

    model.compile(loss = 'mean_absolute_percentage_error', optimizer = optimizer) # Loss는 Metric과 가장 유사한 MAE 사용
    
    model.fit(X_t,y_t, epochs=20, verbose=1)
    
    pred = model.predict(X_test_t)
    
    return pred, model

# my model
def my_model(X_test_t,X_t,y_t):
    activation = 'elu'
    padding = 'causal'
    model = Sequential()
    model.add(keras.layers.InputLayer((6, 1)))
    model.add(Conv1D(filters = 4, kernel_size = 3, padding = padding))
    model.add(BatchNormalization()) # 비선형 성질을 유지 하면서 학습 될 수 있게 해주고, regularization 효과를 가지기 위해 배치정규화 설정
    model.add(Activation(activation = activation))
    model.add(Dropout(0.2))
    
    model.add(keras.layers.InputLayer((6, 1)))
    model.add(Conv1D(filters = 16, kernel_size = 3, padding = padding))
    model.add(BatchNormalization()) # 비선형 성질을 유지 하면서 학습 될 수 있게 해주고, regularization 효과를 가지기 위해 배치정규화 설정
    model.add(Activation(activation = activation))
    model.add(Dropout(0.2))
    
    model.add(Conv1D(filters = 32, kernel_size = 3, padding = padding))
    model.add(BatchNormalization()) # 비선형 성질을 유지 하면서 학습 될 수 있게 해주고, regularization 효과를 가지기 위해 배치정규화 설정
    model.add(Activation(activation = activation))
    model.add(Dropout(0.2))
    
    model.add(Conv1D(filters = 64, kernel_size = 3, padding = padding))
    model.add(BatchNormalization()) # 비선형 성질을 유지 하면서 학습 될 수 있게 해주고, regularization 효과를 가지기 위해 배치정규화 설정
    model.add(Activation(activation = activation))
    model.add(Dropout(0.2))
    
    model.add(Conv1D(filters = 128, kernel_size = 3, padding = padding))
    model.add(BatchNormalization()) # 비선형 성질을 유지 하면서 학습 될 수 있게 해주고, regularization 효과를 가지기 위해 배치정규화 설정
    model.add(Activation(activation = activation))
    model.add(Dropout(0.2))
    
    # Convolution 연산을 진행한 결과를 다시 Flatten 하여 최종 값(Dense(1)) 도출
    model.add(Flatten())
    model.add(Dense(128, activation = activation)) 
    model.add(Dense(64, activation = activation))
    model.add(Dense(32, activation = activation)) 
    model.add(Dense(16, activation = activation)) 
    model.add(Dense(4, activation = activation)) 
    model.add(Dense(1))# output size
    
    # Optimization의 경우 RMSprop으로 선택(시계열의 경우 RMSprop이 조금 더 우수한 성능을 보임)
    optimizer = keras.optimizers.RMSprop() # 실제로 RMSprop, SGD, Adam 등을 비교해본 결과 RMSprop가 가장 우수했음

    model.compile(loss = 'mean_absolute_percentage_error', optimizer = optimizer) # Loss는 Metric과 가장 유사한 MAE 사용
    
    model.fit(X_t,y_t, epochs=20, verbose=1)
    
    pred = model.predict(X_test_t)
    
    return pred, model

In [ ]:
dev_pred, dev_model = set_model(X_test_t, X_t, y_t)

https://dacon.io/competitions/official/235720/codeshare/3746?page=1&dtype=recent

In [ ]:
cnn_pred, cnn_model = CNN(X_test_t,X_t,y_t)

In [ ]:
lstm_pred, lstm_model = LSTM(X_test_t,X_t,y_t)

In [ ]:
lstm_pred.shape, lstm_pred

In [ ]:
submission['공급량'] = np.round(lstm_pred,3)
submission.to_csv('[D-13]lstm.csv',index=False)
submission

In [ ]:
DNN_pred, DNN_model = DNN(X_test,X,y)

In [ ]:
DNN_pred.shape, DNN_pred

In [ ]:
DNN_pred = np.round(DNN_pred,3)
submission['공급량'] = DNN_pred
submission.to_csv('[D-16]dnn.csv',index=False)

In [ ]:
import itertools
Bidirectlstm_pred_list = list(itertools.chain(*Bidirectlstm_pred))
Bidirectlstm_pred_all = []
for i in Bidirectlstm_pred_list:
    Bidirectlstm_pred_all.append(np.round(i,3))
Bidirectlstm_pred_all = np.array(Bidirectlstm_pred_all)
Bidirectlstm_pred_all

In [ ]:
Bidirectlstm_pred_all

In [ ]:
submission['공급량'] = Bidirectlstm_pred_flatten
submission.to_Csv("Bidirectlstm.csv",index=False)
submission

https://www.kaggle.com/mavillan/lightgbm-quantile-regression

In [ ]:
import itertools
lstm_pred_list = list(itertools.chain(*lstm_pred))
lstm_pred_all = []
for i in lstm_pred_list:
    lstm_pred_all.append(np.round(i,3))
lstm_pred_all = np.array(lstm_pred_all)

In [ ]:
submission['공급량'] = lstm_pred_all
submission.to_csv("lstm_baseline.csv",index=False)

In [ ]:
import itertools
cnn_pred_list = list(itertools.chain(*cnn_pred))
cnn_pred_all = []
for i in cnn_pred_list:
    cnn_pred_all.append(np.round(i,3))
cnn_pred_all = np.array(cnn_pred_all)

In [ ]:
submission['공급량'] = cnn_pred_all
submission.to_csv('cnn_pred.csv',index=False)

random_seed 고정하는방법 찾아보기
하이퍼 파라미터

In [ ]:
#cat_best = {'subsample': 0.2, 'reg_lambda': 5, 'random_strength': 21, 'od_wait': 893, 'min_data_in_leaf': 1, 'learning_rate': 0.09, 'leaf_estimation_method': 'Newton', 'leaf_estimation_iterations': 1, 'iterations': 827, 'bootstrap_type': 'MVS'}
#lgb_best = {'subsample': 1.0, 'random_state': 2020, 'n_estimators': 358, 'max_depth': None, 'learning_rate': 0.1, 'lambda': 2, 'colsample_bytree': 1.0, 'cat_smooth': 45, 'alpha': 5}
#xgb_best = {'subsample': 0.8, 'random_state': 21, 'n_estimators': 346, 'max_depth': None, 'learning_rate': 0.1, 'lambda': 5, 'colsample_bytree': 1.0, 'alpha': 10}

#lgb_kaggle = {'max_bin':200, 'bagging_fraction':0.75, 'bagging_freq':5, 'bagging_seed':7, 'feature_fraction':0.2, 'feature_fraction_seed':2020, 'verbose':-1, 'random_state':2020, 'n_estimators':400, 'learning_rate':0.1}

#lgb_ga = {'boosting_type': 'gbdt', 'n_jobs': -1, 'num_leaves': 77, 'learning_rate': 0.38841071071071076, 'colsample_bytree': 0.7907907907907907, 'subsample': 0.8037037037037037, 'n_estimators': 202, 'min_child_weight': 1.201210960960961, 'min_child_samples': 39, 'reg_alpha': 0.40900000000000003, 'reg_lambda': 0.3418734734734735, 'random_state': 42}

#model0 = KNeighborsRegressor()

#model1 = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =42)

#model2 = LGBMRegressor(boosting_type='gbdt',objective="regression",metric="mae",random_state=42) # ,feature_fraction=0.38,bagging_fraction=0.41,bagging_seed=42
#model2 = LGBMRegressor(n_estimators = 1000, random_state = 21, reg_alpha = 5.2651567238270305, reg_lambda = 0.6105126117528492, colsample_bytree = 0.7, subsample = 0.6, learning_rate = 0.008, max_depth = 3, num_leaves = 286, min_child_samples = 171, min_data_per_groups = 13)

# Bernoulli, Bayesian
model3 = CatBoostRegressor(learning_rate = 0.1, bootstrap_type = 'Bernoulli')
#model3 = CatBoostRegressor(iterations = 500, od_wait = 1092, learning_rate = 0.4111333546053877, reg_lambda = 42.90680713323128, subsample = 0.3943513547454773, random_strength = 44.68346376029576, depth = 2, min_data_in_leaf = 12, leaf_estimation_iterations = 4)

model4 = BaggingRegressor()

model5 = ExtraTreesRegressor()

model6 = DecisionTreeRegressor()

model7 = ExtraTreeRegressor()

#model8 = XGBRegressor(learning_rate = 0.1, metric = MAE, random_state = 42)
#model8 = XGBRegressor(alpha = 0.01643163148521856, colsmaple_bytree = 0.3, subsample = 0.4, learning_rate = 0.03, n_estimators = 500, max_depth = 11, random_state = 42, min_child_weight = 181, reg_lambda = 0.4497515909263729,)

#model2 = LGBMRegressor(reg_alpha=0.878075,colsample_bytree=0.3,reg_lambda=0.004216,learning_rate=0.01,max_depth=7,min_child_weight=272,n_estimators=1000,random_state=2020,subsample=0.8)
#model3 = CatBoostRegressor(depth=10,iterations=2000,leaf_estimation_iterations=10,learning_rate=0.608965,min_data_in_leaf=20,od_wait=1559,random_strength=38.042741,reg_lambda=98.344238,subsample=0.998485)
#model8 = XGBRegressor(alpha = 0.01643163148521856, colsmaple_bytree = 0.3, subsample = 0.4, learning_rate = 0.03, n_estimators = 500, max_depth = 11, random_state = 42, min_child_weight = 181, reg_lambda = 0.4497515909263729,)

model2 = LGBMRegressor()
#model3 = CatBoostRegressor(**cat_best)
model8 = XGBRegressor(learning_rate = 0.1, metric = MAE, random_state = 42)

#model9 = NGBRegressor(natural_gradient=True, learning_rate = 0.03,Base=default_tree_learner, Dist=Normal, Score=MLE)

#model10 = MLPRegressor()

#model11 = AdaBoostRegressor()

model12 = HistGradientBoostingRegressor()

meta_model = RandomForestRegressor()

In [ ]:
models = [model3, model4, model5, model6, model7, model8, model12, meta_model] #, model2

In [ ]:
estimators = [('cat', model3), ('bg', model4), ('ex', model5), ('dt', model6), ('et', model7), ('xgb',model8), ('hb', model12), ('rf',meta_model)] #, ('lgb', model2),

In [ ]:
stacktransformer = StackingTransformer(estimators, regression=True, shuffle=True, verbose=2)
stacktransformer = stacktransformer.fit(X,y_log)
S_train = stacktransformer.transform(X)
S_test = stacktransformer.transform(X_test)

In [ ]:
S_train.shape,X.shape

In [ ]:
S_test.shape,X_test.shape

In [ ]:
model12.fit(S_train,y_log)
pred = model2.predict(S_test)
pred_log = np.expm1(pred)

In [ ]:
stack = StackingCVRegressor(regressors=models,meta_regressor=model2,use_features_in_secondary=True,n_jobs=-1) #,use_features_in_secondary=True,n_jobs=-1

In [ ]:
stack.fit(X, y_log)
# stack.fit(X_poly, y, sample_weight=weight)

https://www.kaggle.com/harangdev/skewed-feature-target-log

https://www.kaggle.com/raoulma/ny-stock-price-prediction-rnn-lstm-gru

https://www.kaggle.com/ashishpatel26/lstm-demand-forecasting

https://www.kaggle.com/rajmehra03/bike-sharing-demand-rmsle-0-3194

In [ ]:
X_test.shape

In [ ]:
stack_pred = stack.predict(X_test) # X_test_poly
stack_pred_log = np.expm1(stack_pred)

# https://teddylee777.github.io/scikit-learn/scikit-learn-ensemble

In [ ]:
"""stack.fit(x_train,y_train)
pred = stack.predict(x_val)
pred = pd.Series(pred)
pred.index = np.arange(y_val.index[0], y_val.index[-1]+1)
plot_series(y_train, y_val, pd.Series(pred), markers=[',' , ',', ','])
plt.title(str(stack) + ' / mae: ' + str(paul_MAE(y_val,pred)))"""

In [ ]:
"""meta_model.fit(x_train,y_train)
pred = mata_model.predict(x_val)
pred = pd.Series(pred)
pred.index = np.arange(y_val.index[0], y_val.index[-1]+1)
plot_series(y_train, y_val, pd.Series(pred), markers=[',' , ',', ','])
plt.title(str(meta_model) + ' / mae: ' + str(paul_MAE(y_val,pred)))"""

In [ ]:
stack_pred_log
submission['공급량'] = np.round(stack_pred_log,3)
submission.to_csv('[restart1]stack_pred_log.csv',index=False)

In [ ]:
submission

In [ ]:
pred_log
submission['공급량'] = np.round(pred_log,3)
submission.to_csv('[restart1]pred_log.csv',index=False)

In [ ]:
submission

In [ ]:
import itertools
lstm_pred_list = list(itertools.chain(*lstm_pred))
lstm_pred_all = []
for i in lstm_pred_list:
    lstm_pred_all.append(np.round(i,3))
lstm_pred_all = np.array(lstm_pred_all)
lstm_pred_all

In [ ]:
#esemble_pred = (stack_pred_log + pred_log + lstm_pred_all)/3
ensemble_pred0 = stack_pred_log * 0.25 + pred_log * 0.35 + lstm_pred_all * 0.4
ensemble_pred0

In [ ]:
esemble_pred1 = (stack_pred_log + pred_log + lstm_pred_all)/3
esemble_pred1

In [ ]:
submission['공급량'] = np.round(ensemble_pred0, 3)
submission.to_csv('[D-16]3en_w.csv',index=False)

In [ ]:
submission

# 연도별 상승분 반영 https://dacon.io/competitions/official/235837/codeshare/3724?page=1&dtype=recent

histgbm https://machinelearningmastery.com/gradient-boosting-with-scikit-learn-xgboost-lightgbm-and-catboost/

특정 요소에 첫번째, 마지막 날짜 구하기
https://icodebroker.tistory.com/6861?category=857192

https://dacon.io/competitions/official/235683/codeshare/2340?page=1&dtype=recent
https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/

nueral_network https://frhyme.github.io/python-lib/is_mlp_regressor_good/

In [ ]:
def blended_predictions(models,stack_pred,weight,X_test):
    model_0 = models[0].fit(X_poly, y)
    model_1 = models[1].fit(X_poly, y)
    model_6 = models[6].fit(X_poly, y)
    
    return ((weight[0] * model_0.predict(X_test)) + \
            (weight[1] * model_1.predict(X_test)) + \
            (weight[6] * model_6.predict(X_test)) + \
            (weight[7] * stack_pred))


weights = [0.2,0.15,0.15,0.5]
blended_pred = blended_predictions(models,stack_pred,weight,X_test_poly)
blended_pred

# https://dining-developer.tistory.com/24

In [ ]:
blended_pred_expm1 = np.expm1(blended_pred)
blended_pred_expm1

In [ ]:
#explainer = shap.Explainer(model)
#shap_values = explainer(X_test)
#model = stack
#model.fit(X_poly, y, sample_weight=weight)
#model.fit(X,y)

explainer = shap.TreeExplainer(stack)
shap_values = explainer.shap_values(X)

shap_sum = np.abs(shap_values).mean(axis=0)
importance_df = pd.DataFrame(
    [X_test.columns.tolist(), shap_sum.tolist()]).T
importance_df.columns = ['column_name', 'shap_importance']
importance_df = importance_df.sort_values('shap_importance', ascending=False)

In [ ]:
importance_df

In [ ]:
explainer = shap.TreeExplainer(stack)
shap_values = explainer(X_test_poly)

shap.summary_plot(shap_values, X_test_poly)

In [ ]:
shap.summary_plot(shap_values, X_test_poly, plot_type='bar')

In [ ]:
shap_selected_features = list(
    importance_df[importance_df['shap_importance'] > 17]['column_name'])

In [ ]:
shap_selected_features

In [ ]:
X = X[shap_selected_features]
X_test = X_test[shap_selected_features]

In [ ]:
pred = model.predict(x_val)
pred = pd.Series(pred)
pred.index = np.arange(y_val.index[0], y_val.index[-1]+1)
plot_series(y_train, y_val, pd.Series(pred), markers=[',' , ',', ','])

print('best iterations: {}'.format(model.best_iteration))
print('SMAPE : {}'.format(SMAPE(y_val, pred)))
print('nmae:{}'.format(nmae(y_val,pred)))

# https://gorakgarak.tistory.com/1278
# https://towardsdatascience.com/build-an-extreme-learning-machine-in-python-91d1e8958599
# ELM model
#https://www.kaggle.com/ghazalkheyroddin/feature-selection-using-shap
#https://dacon.io/competitions/official/235713/codeshare/2767
#https://dacon.io/competitions/official/235743/codeshare/2974
#https://www.kaggle.com/lqdisme/house-prices-prediction-stacking-models
#https://pythonrepo.com/repo/vecxoz-vecstack-python-machine-learning
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html#sphx-glr-auto-examples-model-selection-plot-underfitting-overfitting-py
# https://www.kaggle.com/kabure/extensive-eda-and-modeling-xgb-hyperopt